# Loop 2 Analysis: Ensemble Strategy & Gap Analysis

## Key Questions:
1. What is the best score achievable by ensembling all available public submissions?
2. Which N values have the most room for improvement?
3. What techniques haven't been tried yet?

In [ ]:
import os
import glob
import math
import pandas as pd
import numpy as np
from numba import njit
from tqdm import tqdm

# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs,ys,degs,tx,ty):
    n=xs.size; V=tx.size
    mnx=1e300; mny=1e300; mxx=-1e300; mxy=-1e300
    for i in range(n):
        r=degs[i]*math.pi/180.0
        c=math.cos(r); s=math.sin(r)
        xi=xs[i]; yi=ys[i]
        for j in range(V):
            X=c*tx[j]-s*ty[j]+xi
            Y=s*tx[j]+c*ty[j]+yi
            if X<mnx: mnx=X
            if X>mxx: mxx=X
            if Y<mny: mny=Y
            if Y>mxy: mxy=Y
    side=max(mxx-mnx,mxy-mny)
    return side*side/n

def strip(a):
    return np.array([float(str(v).replace("s","")) for v in a],np.float64)

tx, ty = make_polygon_template()
print("Polygon template loaded")

In [ ]:
# Find all CSV files
csv_files = []
for root, dirs, files in os.walk('/home/code/datasets'):
    for f in files:
        if f.endswith('.csv'):
            csv_files.append(os.path.join(root, f))

# Also check experiments
for root, dirs, files in os.walk('/home/code/experiments'):
    for f in files:
        if f.endswith('.csv'):
            csv_files.append(os.path.join(root, f))

print(f"Found {len(csv_files)} CSV files")
for f in csv_files:
    print(f"  {f}")

In [ ]:
# Score all submissions and find best per N
best = {n:{"score":1e300,"data":None,"src":None} for n in range(1,201)}

for fp in tqdm(csv_files, desc="Scanning"):
    try:
        df = pd.read_csv(fp)
    except Exception:
        continue
    if not {"id","x","y","deg"}.issubset(df.columns):
        continue
    df = df.copy()
    df["N"] = df["id"].astype(str).str.split("_").str[0].astype(int)
    for n,g in df.groupby("N"):
        if n<1 or n>200:
            continue
        xs = strip(g["x"].to_numpy())
        ys = strip(g["y"].to_numpy())
        ds = strip(g["deg"].to_numpy())
        sc = score_group(xs,ys,ds,tx,ty)
        if sc < best[n]["score"]:
            best[n]["score"]=float(sc)
            best[n]["data"]=g.drop(columns=["N"]).copy()
            best[n]["src"]=fp.split('/')[-1]

print("\nBest scores found per source:")

In [ ]:
# Calculate ensemble score
ensemble_score = sum(best[n]["score"] for n in range(1, 201))
print(f"\nEnsemble score (best per N): {ensemble_score:.6f}")
print(f"Current best: 70.734327")
print(f"Target: 68.931058")
print(f"Gap to target: {ensemble_score - 68.931058:.6f}")

# Count sources
source_counts = {}
for n in range(1, 201):
    src = best[n]["src"]
    source_counts[src] = source_counts.get(src, 0) + 1

print("\nSource distribution:")
for src, count in sorted(source_counts.items(), key=lambda x: -x[1]):
    print(f"  {src}: {count} N values")

In [ ]:
# Identify worst N values (highest contribution to score)
contributions = []
for n in range(1, 201):
    contrib = best[n]["score"]
    contributions.append((n, contrib, best[n]["src"]))

contributions.sort(key=lambda x: -x[1])

print("\nTop 30 worst N values (highest contribution):")
print(f"{'N':>4} {'Contribution':>12} {'Source':>30}")
print("-" * 50)
for n, contrib, src in contributions[:30]:
    print(f"{n:4d} {contrib:12.6f} {src:>30}")

In [ ]:
# Calculate potential improvement if we could improve worst N values
print("\nPotential improvement analysis:")
print("If we improve the worst 20 N values by X%:")
worst_20_total = sum(c[1] for c in contributions[:20])
for pct in [5, 10, 15, 20, 25]:
    improvement = worst_20_total * (pct / 100)
    new_score = ensemble_score - improvement
    print(f"  {pct}% improvement: {ensemble_score:.4f} -> {new_score:.4f} (saves {improvement:.4f})")

print(f"\nTo reach target {68.931058}, need to save: {ensemble_score - 68.931058:.4f}")
print(f"Worst 20 contribute: {worst_20_total:.4f} ({100*worst_20_total/ensemble_score:.1f}% of total)")

In [ ]:
# Save ensemble submission
rows = []
for n in range(1, 201):
    if best[n]["data"] is not None:
        rows.append(best[n]["data"])

ensemble_df = pd.concat(rows, ignore_index=True)
ensemble_df["sn"] = ensemble_df["id"].str.split("_").str[0].astype(int)
ensemble_df["si"] = ensemble_df["id"].str.split("_").str[1].astype(int)
ensemble_df = ensemble_df.sort_values(["sn","si"]).drop(columns=["sn","si"])
ensemble_df = ensemble_df[['id','x','y','deg']]

ensemble_path = '/home/code/experiments/002_extended_optimization/ensemble_best.csv'
ensemble_df.to_csv(ensemble_path, index=False)
print(f"Saved ensemble to {ensemble_path}")
print(f"Ensemble score: {ensemble_score:.6f}")

In [ ]:
# Compare with current best
current_best = 70.734327
print(f"\n=== SUMMARY ===")
print(f"Current best LB: {current_best:.6f}")
print(f"Ensemble score: {ensemble_score:.6f}")
print(f"Improvement from ensemble: {current_best - ensemble_score:.6f}")
print(f"Target: 68.931058")
print(f"Gap remaining: {ensemble_score - 68.931058:.6f}")

if ensemble_score < current_best:
    print(f"\n*** ENSEMBLE IMPROVES SCORE! ***")
else:
    print(f"\n*** Ensemble does NOT improve - current submission is already best for all N ***")